In [ ]:
import pyspark.sql.functions as f

So far, you've learned about visualizing PySpark DataFrame properties, data cleaning using PySpark, and how to create new features.

Now it's time to apply this knowledge to a new dataset.

In this notebook, you'll work with the `product-supplier` dataset that we've downloaded along with the `orders` dataset. 

The file is saved in the dbfs at `/FileStore/lp-big-data/orders-data/product-supplier.csv`.

1. Load the dataset into a spark dataframe and display it

In [ ]:
df_products = spark.read.csv(
    "/FileStore/lp-big-data/orders-data/product-supplier.csv",
    header=True,
    inferSchema=True,
    sep=",",
)

# OR

df_products = (
    spark.read.format('csv')
    .option('inferSchema', 'true')
    .option('header', 'true')
    .option('sep', ',')
    .load('/FileStore/lp-big-data/orders-data/product-supplier.csv')
)

df_products.display()

2. Print the schema

In [ ]:
df_products.printSchema()

3. How many observations does the dataset contain?

In [ ]:
df_products.count()

4. Rename the columns to standardize them.

- Lower case all column names
- Replace the spaces with underscores `_`

In [ ]:
# Option 1: Using the toDF() method specifying all the column names
df_products_renamed = (
    df_products
    .toDF('product_id',
        'product_line',
        'product_category',
        'product_group',
        'product_name',
        'supplier_country',
        'supplier_name',
        'supplier_id',
        'announcement_date',
        'launch_date',
    )
)

df_products_renamed.display()

In [ ]:
# Option 2: Using a for loop and pure python to create
# new column names and using the toDF() method to rename the columns

# Get a list of strings with the column names
column_names_list = df_products.columns

# Create an empty list to save the new column names
column_names_new = []

# Loop through the column names list
for col_name in column_names_list:
    # Lowercase the column name and replace spaces with underscores
    column_names_new.append(col_name.lower().replace(' ', '_'))

# Rename the columns
df_products_renamed = df_products.toDF(*column_names_new)

5. Check if there are missing values

In [ ]:
df_products_renamed.describe().display()

Replace the missing categories by 'Unknown'

In [ ]:
df_products_filled = (
    df_products_renamed
    .fillna('Unknown', subset=(['product_category']))
)

In [ ]:
# Check that the missing values have been filled
df_products_filled.describe().display()

6. What are the names of all possible product lines?

In [ ]:
# Option 1: Get them visually in a table, along with the count of each product line
df_products_filled.groupBy('product_line').count().display()

In [ ]:
# Option 2: Using the rdd.flatMap().collect() method to get the product lines in a list
df_products_filled.select('product_line').rdd.flatMap(lambda x: x).collect()

In [ ]:
# Option 3: Selecting the distinct values, transform the resulting
# pySpark DataFrame into a pandas DataFrame and use the .values attribute
df_products_filled.select('product_line').distinct().toPandas().values

7. Create a new column called `lead_time` with the amount of time in days between each product's announcement and launch dates

Hint: Check out the [DateTime functions](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#datetime-functions) from the PySpark SQL library

In [ ]:
df_products_lead = (
    df_products_filled
    .withColumn(
        'lead_time',
        f.datediff(f.col('launch_date'), f.col('announcement_date'))
    )
)

df_products_lead.display()

8. The announcement and launch dates are in UTC format. Convert them to New York's timezone.

In [ ]:
df_products_timezone = (
    df_products_lead
    .withColumn('announcement_date', f.from_utc_timestamp(f.col('announcement_date'), 'America/New_York'))
    .withColumn('launch_date', f.from_utc_timestamp(f.col('launch_date'), 'America/New_York'))
)

df_products_timezone.display()

9. Create a new column with the suppliers' continent

In [ ]:
# Let's see the possible values for the supplier_country column and their counts
df_products_timezone.groupBy('supplier_country').count().display()

In [ ]:
df_products_preprocessed = (
    df_products_timezone
    # Define to which continent the supplier belongs based on the country
    .withColumn(
        'supplier_continent',
        f.when(
            f.col('supplier_country').isin(
                ['NL', 'PT', 'GB', 'DE', 'ES', 'FR', 'NO', 'DK', 'BE', 'SE']
                ),
            'Europe'
        )
        .when(f.col('supplier_country').isin(['CA', 'US']), 'America')
        .when(f.col('supplier_country').isin(['AU']), 'Oceania')
        .otherwise('Unkown')
    )
)

df_products_preprocessed.display()

10. Save the data to the dbfs to the folder `/FileStore/lp-big-data/orders-data/` under the name `products-preprocessed.csv`.

In [ ]:
df_products_preprocessed.write.csv('/FileStore/lp-big-data/orders-data/products-preprocessed.csv', header=True)